# Data Extraction

In [1]:
import pandas as pd
import seaborn as sns
import requests
from tqdm import trange

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

#initial url
url = 'https://www.the-numbers.com/movie/budgets/all'
html = requests.get(url, headers=headers).content
table_list = pd.read_html(html)

#specified url only has 1 table
assert len(table_list) == 1

movie_data = table_list[0]

#specified url only has 1 table
assert len(table_list) == 1
movie_data.head(2)

,Unnamed: 0,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,1,"Dec 9, 2022",Avatar: The Way of Water,"$460,000,000","$657,411,991","$2,243,117,055"
1,2,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,794,731,755"


In [3]:
#get all pages worth of data. There are 6301 entries with 99 entries on each page

for entry_index in trange(101,6401,100):
    next_page_url = url + f"/{str(entry_index)}"

    html = requests.get(next_page_url, headers=headers).content
    table_list = pd.read_html(html)

    assert len(table_list) == 1

    page_data = table_list[0]

    #add to main dataframe

    movie_data = pd.concat([movie_data,page_data], ignore_index=True)
    

100%|██████████| 63/63 [01:25<00:00,  1.35s/it]


In [4]:
movie_data.shape

(6364, 6)

The shape of the data is as expected. Lets do some EDA.

In [5]:
movie_data.head()

,Unnamed: 0,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,1,"Dec 9, 2022",Avatar: The Way of Water,"$460,000,000","$657,411,991","$2,243,117,055"
1,2,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,794,731,755"
2,3,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
3,4,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
4,5,"May 17, 2023",Fast X,"$340,000,000",$0,$0


In [6]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6364 entries, 0 to 6363
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         6364 non-null   int64 
 1   Release Date       6364 non-null   object
 2   Movie              6364 non-null   object
 3   Production Budget  6364 non-null   object
 4   Domestic Gross     6364 non-null   object
 5   Worldwide Gross    6364 non-null   object
dtypes: int64(1), object(5)
memory usage: 298.4+ KB


There are no null columns, so no need to drop or fill data on the index axis. That being said, the "Unnamed" column will need to be dropped. In terms of datatypes, lets convert the release date to a datettime and the financial numbers to int.

In [7]:
movie_data.drop(columns=["Unnamed: 0"], inplace = True)
movie_data.head()

,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,"Dec 9, 2022",Avatar: The Way of Water,"$460,000,000","$657,411,991","$2,243,117,055"
1,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,794,731,755"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
3,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
4,"May 17, 2023",Fast X,"$340,000,000",$0,$0


Lets explore and convert the release date columns

In [8]:
movie_data["Release Date"].value_counts()

Unknown         113
Oct 19, 2007      9
Oct 24, 2008      9
Oct 8, 2010       9
Apr 1, 2011       9
               ... 
Sep 16, 1994      1
Jan 26, 2001      1
Jan 28, 2013      1
Feb 13, 2013      1
Mar 2, 2021       1
Name: Release Date, Length: 3090, dtype: int64

There are 113 rows with an unkown date. I could find out when they were released, but its a small number of rows to drop in this large dataset. There are also rows where only the year has been noted. Although the full date format is not required, it is a nice to have. Rows where the release date is less than 5 characters will be dropped too.

In [9]:
movie_data_clean = movie_data[(movie_data["Release Date"] != "Unknown") &
                                    (movie_data["Release Date"].str.len() > 9)]
print(movie_data_clean.shape)
movie_data_clean.head()

(6218, 5)


,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,"Dec 9, 2022",Avatar: The Way of Water,"$460,000,000","$657,411,991","$2,243,117,055"
1,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,794,731,755"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
3,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
4,"May 17, 2023",Fast X,"$340,000,000",$0,$0


In [12]:
movie_data_final = pd.DataFrame()
movie_data_final["Release Date"] = pd.to_datetime(movie_data_clean["Release Date"], format="%b %d, %Y")
movie_data_final.head()

,Release Date
0,2022-12-09
1,2019-04-23
2,2011-05-20
3,2015-04-22
4,2023-05-17


Lets keep the movie name column as is, but change the comma seperated currency value to an integer amount

In [17]:
movie_data_final["Title"] = movie_data_clean["Movie"]

movie_data_final["Production Budget"] = movie_data_clean["Production Budget"].str.replace("$","")
movie_data_final["Production Budget"] = movie_data_final["Production Budget"].str.replace(",","")

movie_data_final["Domestic Gross"] = movie_data_clean["Domestic Gross"].str.replace("$","")
movie_data_final["Domestic Gross"] = movie_data_final["Domestic Gross"].str.replace(",","")

movie_data_final["Worldwide Gross"] = movie_data_clean["Worldwide Gross"].str.replace("$","")
movie_data_final["Worldwide Gross"] = movie_data_final["Worldwide Gross"].str.replace(",","")

movie_data_final.head()

C:\Users\Sid Murthy\AppData\Local\Temp\ipykernel_6936\3410810997.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_data_final["Production Budget"] = movie_data_clean["Production Budget"].str.replace("$","")
C:\Users\Sid Murthy\AppData\Local\Temp\ipykernel_6936\3410810997.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_data_final["Domestic Gross"] = movie_data_clean["Domestic Gross"].str.replace("$","")
C:\Users\Sid Murthy\AppData\Local\Temp\ipykernel_6936\3410810997.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

,Release Date,Title,Production Budget,Domestic Gross,Worldwide Gross
0,2022-12-09,Avatar: The Way of Water,460000000,657411991,2243117055
1,2019-04-23,Avengers: Endgame,400000000,858373000,2794731755
2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,379000000,241071802,1045713802
3,2015-04-22,Avengers: Age of Ultron,365000000,459005868,1395316979
4,2023-05-17,Fast X,340000000,0,0


Now that the data is cleaned and formatted, we are ready to write it to a csv file

In [18]:
movie_data_final.to_csv("../data/movie_data.csv")